In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import (
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    pipeline, DataCollatorForSeq2Seq, EarlyStoppingCallback
)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from safetensors.torch import load_file

In [10]:
models=[
    "/kaggle/input/m2m-eng-naga-weights-extra-data/m2m_model",
    "/kaggle/input/nllb-eng-naga-checkpoint-extra-data/nllb_model",
    "/kaggle/input/mbart-eng-naga-checkpoint-extra-data/mbart_model"
]
source_texts=[
    # "Why are all these people gathering at the river?",
    "Every tree which does not produce good fruit will be cut down and thrown into the fire.",
    # "Peter, let's sail to deeper water and put your nets out there.",
    "Master, we've worked hard all night and havent caught anything.",
    # "Do to others as you want them to do to you",
    "Do good secretly, not to attract attention",
    # "The eye is the light of the body. If your eye is clear then your whole body will be illuminated. But if your eye is cloudy, you live in darkness.",
    # "Philip, how are all these people going to get food? Why don't you give them something?",
    # "Here is a boy with five loaves and two fishes. That's all there is.",
    # "Are you looking for me because of the bread I gave you?",
    # "We could have sold it. We could have given the money to the poor.",
    # "Listen to that. Are we not good enough?",
    # "What a disaster! The wine has run out!",
    "What's happening up there?",
    # "Did you hear that? How dare he say that!",
    "If you had been here my brother would not have died",
    # "Look how much he loved him.",
    # "But couldn't he have prevented his death?",
    "Just think! It's better that one man dies for the nation than that all are destroyed.",
    "One of you is going to betray me.",
    # "Do you have anything to eat?",
    # "No, we haven't caught a thing!",
    # "Throw your nets out on the other side!"
    
]
reference_texts=[
    # "Kiley sob manu khan nodi te jamma hoishe?",
    "Jun gaas bhal phol nedea etu ke ka deya hobo aru jui te phelai dibo!",
    # "Peter, pani besi thaka jag ate jabo aro tumi laga jaal tatey phelabi.",
    "Malik, raat bhor jaal marise hoilebi olopbi maas panai.",
    # "Tumi jenika manu para bhal lobole mon enika hae tumi be manu ke debi.",
    "Lukai kena bhal kuribi manulaga osorte dekhai kena nohoi.",
    # "Chuku tu gao laga puhor ase. Jodi tumilaga chuku safa thake koile tumi laga sob gao tu safa ase. Hoilebi Jodi tumi laga chuku moila ase koile, tumi andhera te thaki ase.",
    # "Philip, etu manu khan karne kineka kurikena bhat khelabo? Keelea khabole keba nadea?",
    # "Yatae ekta chokra pasta roti aro duita choto maas loikena asae. Etu hae asae.",
    # "Moi tumi ke roti diya karne ke Tumi moike bisari asae?",
    # "Moikhan poisa u gorib khan ke diya tu hae bhal hubo.",
    # "ke moi khan bhal jivan betai thakia khan kebe ka kena phalabo?",
    # "Ki hoishe! Modu khotom hoishe!",
    "Upor te ki hoiase?",
    # "Kineka Tai tu! Tumi etu hunishe?",
    "Jodi apuni yatae thakisae koile moilaga Bhai muribo thakeyanai!",
    # "Ena sabi; Tai hosabi taike morom kore.",
    # "Hoilebi taitu Lazarus laga mura din tu ruka bole para nai?",
    "Nejor laga dimak chola bi! Pura desh beya hua bodli ekjon ekla hae mora tu bhal asae.",
    "Tumi khan para ekjon moike dhoka dibo asae.",
    # "Ki tumi khan logote khabo lae eku nai ?",
    # "Nai, amikhan ekubi dhuri bole parianai!",
    # "Jaal tu tumi khan laga dusra phale phelabi."

]

for j,i in enumerate(models):
    tokenizer = AutoTokenizer.from_pretrained(i)
    model = AutoModelForSeq2SeqLM.from_pretrained(i)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("--------------------------------------------------------------")
    model = model.to(device)
    for n in range(len(source_texts)):
        inputs = tokenizer(source_texts[n], return_tensors="pt", max_length=128, truncation=True).to(device)
        
        # Generate output token IDs
        with torch.no_grad():
            generated = model.generate(**inputs, max_length=128)
            translated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
        
        print(f"Model: {model_names[j]}")
        print(f"Translated: {translated_text}")
        print(f"Reference: {reference_texts[n]}")

--------------------------------------------------------------
Model: m2m
Translated: Juntu gash bhal phol nadhure itu gash ke kati kena jui te phelai dibo.
Reference: Jun gaas bhal phol nedea etu ke ka deya hobo aru jui te phelai dibo!
Model: m2m
Translated: Probhu, amikhan hodai rati te bisi dukh pora kaam korise, aru sob kaam khan bandhi kena thakise.
Reference: Malik, raat bhor jaal marise hoilebi olopbi maas panai.
Model: m2m
Translated: Itu nimite bhal kaam lukai kena rakhibi, aru nijor laga kotha namatibi.
Reference: Lukai kena bhal kuribi manulaga osorte dekhai kena nohoi.
Model: m2m
Translated: Tate ki hoise?
Reference: Upor te ki hoiase?
Model: m2m
Translated: Jodi apni amilaga bhai yate thakise koile amilaga bhai namoribo asele
Reference: Jodi apuni yatae thakisae koile moilaga Bhai muribo thakeyanai!
Model: m2m
Translated: Bhabona koribi! Ekjon manu itu jati nimite moribole he bhal ase, juntu pora sob manu khotom hoi jai.
Reference: Nejor laga dimak chola bi! Pura desh beya

In [7]:
models=[
    "/kaggle/input/m2m-naga-eng-weights-extra-data/m2m_model",
    "/kaggle/input/nllb-naga-eng-checkpoint-extra-data/nllb_model",
    "/kaggle/input/mbart-naga-eng-checkpoint/mbart_model"
]
model_names=["m2m","nllb","mbart"]
reference_texts=[
    # "Why are all these people gathering at the river?",
    # "Every tree which does not produce good fruit will be cut down and thrown into the fire.",
    # "Peter, let's sail to deeper water and put your nets out there.",
    # "Master, we've worked hard all night and havent caught anything.",
    # "Do to others as you want them to do to you",
    # "Do good secretly, not to attract attention",
    # "The eye is the light of the body. If your eye is clear then your whole body will be illuminated. But if your eye is cloudy, you live in darkness.",
    # "Philip, how are all these people going to get food? Why don't you give them something?",
    # "Here is a boy with five loaves and two fishes. That's all there is.",
    # "Are you looking for me because of the bread I gave you?",
    # "We could have sold it. We could have given the money to the poor.",
    # "Listen to that. Are we not good enough?",
    # "What a disaster! The wine has run out!",
    # "What's happening up there?",
    # "Did you hear that? How dare he say that!",
    # "If you had been here my brother would not have died",
    # "Look how much he loved him.",
    # "But couldn't he have prevented his death?",
    # "Just think! It's better that one man dies for the nation than that all are destroyed.",
    # "One of you is going to betray me.",
    # "Do you have anything to eat?",
    # "No, we haven't caught a thing!",
    # "Throw your nets out on the other side!"
    
]
source_texts=[
    # "Kiley sob manu khan nodi te jamma hoishe?",
    # "Jun gaas bhal phol nedea etu ke ka deya hobo aru jui te phelai dibo!",
    # "Peter, pani besi thaka jag ate jabo aro tumi laga jaal tatey phelabi.",
    # "Malik, raat bhor jaal marise hoilebi olopbi maas panai.",
    # "Tumi jenika manu para bhal lobole mon enika hae tumi be manu ke debi.",
    # "Lukai kena bhal kuribi manulaga osorte dekhai kena nohoi.",
    # "Chuku tu gao laga puhor ase. Jodi tumilaga chuku safa thake koile tumi laga sob gao tu safa ase. Hoilebi Jodi tumi laga chuku moila ase koile, tumi andhera te thaki ase.",
    # "Philip, etu manu khan karne kineka kurikena bhat khelabo? Keelea khabole keba nadea?",
    # "Yatae ekta chokra pasta roti aro duita choto maas loikena asae. Etu hae asae.",
    # "Moi tumi ke roti diya karne ke Tumi moike bisari asae?",
    # "Moikhan poisa u gorib khan ke diya tu hae bhal hubo.",
    # "ke moi khan bhal jivan betai thakia khan kebe ka kena phalabo?",
    # "Ki hoishe! Modu khotom hoishe!",
    # "Upor te ki hoiase?",
    # "Kineka Tai tu! Tumi etu hunishe?",
    # "Malik, Jodi apuni yatae thakisae koile moilaga Bhai muribo thakeyanai!",
    # "Ena sabi; Tai hosabi taike morom kore.",
    # "Hoilebi taitu Lazarus laga mura din tu ruka bole para nai?",
    # "Nejor laga dimak chola bi! Pura desh beya hua bodli ekjon ekla hae mora tu bhal asae.",
    # "Tumi khan para ekjon moike dhoka dibo asae.",
    # "Ki tumi khan logote khabo lae eku nai ?",
    # "Nai, amikhan ekubi dhuri bole parianai!",
    # "Jaal tu tumi khan laga dusra phale phelabi."

]

for j,i in enumerate(models):
    tokenizer = AutoTokenizer.from_pretrained(i)
    model = AutoModelForSeq2SeqLM.from_pretrained(i)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("--------------------------------------------------------------")
    model = model.to(device)
    for n in range(len(source_texts)):
        inputs = tokenizer(source_texts[n], return_tensors="pt", max_length=128, truncation=True).to(device)
        
        # Generate output token IDs
        with torch.no_grad():
            generated = model.generate(**inputs, max_length=128)
            translated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
        
        print(f"Model: {model_names[j]}")
        print(f"Translated: {translated_text}")
        print(f"Reference: {reference_texts[n]}")

--------------------------------------------------------------
Model: m2m
Translated: Remove your wheat on the other side.
Reference: Throw your nets out on the other side!
--------------------------------------------------------------
Model: nllb
Translated: Go and spread it on the other side.
Reference: Throw your nets out on the other side!
--------------------------------------------------------------
Model: mbart
Translated: Throw the net into your other side.
Reference: Throw your nets out on the other side!


In [ ]:
source_texts=[
    "Every tree which does not produce good fruit will be cut down and thrown into the fire.",
    "Master, we've worked hard all night and havent caught anything.",
    "Do good secretly, not to attract attention",
    "What's happening up there?",
    "If you had been here my brother would not have died",
    "Just think! It's better that one man dies for the nation than that all are destroyed.",
    "One of you is going to betray me.",  
]
reference_texts=[
    "Jun gaas bhal phol nedea etu ke ka deya hobo aru jui te phelai dibo!",
    "Malik, raat bhor jaal marise hoilebi olopbi maas panai.",
    "Lukai kena bhal kuribi manulaga osorte dekhai kena nohoi.",
    "Upor te ki hoiase?",
    "Jodi apuni yatae thakisae koile moilaga Bhai muribo thakeyanai!",
    "Nejor laga dimak chola bi! Pura desh beya hua bodli ekjon ekla hae mora tu bhal asae.",
    "Tumi khan para ekjon moike dhoka dibo asae.",

]